In [1]:
import findspark

findspark.init()

In [33]:
import pyspark.pandas as ps
import numpy as np
import pymongo
import pymysql
import sqlalchemy
import re
from urllib.request import urlretrieve
from tqdm.notebook import tqdm
from PIL import Image

In [4]:
user = 'team04'
pw = '1111'
host = 'ec2-54-95-8-243.ap-northeast-1.compute.amazonaws.com'
db_name = 'resell'
client = pymongo.MongoClient(f'mongodb://{user}:{pw}@{host}:27017/')
db = client.resell

In [9]:
kream = ps.DataFrame(db.kream.find())
kream_detail = ps.DataFrame(db.kream_detail.find())

In [7]:
kream.head()

,_id,brand,kr_name,price,like,post,all_trade,link
0,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,"600,000원",2.6만,"5,572",거래 8.8만,https://kream.co.kr/products/25922
1,Jordan 1 Mid Light Smoke Grey Anthracite,Jordan,조던 1 미드 라이트 스모크 그레이 앤트러사이트,"185,000원",5만,921,거래 3.2만,https://kream.co.kr/products/47257
2,Jordan 1 Low Golf Wolf Grey,Jordan,조던 1 로우 골프 울프 그레이,"238,000원",3만,558,거래 1.4만,https://kream.co.kr/products/48705
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,"299,000원","4,348",128,"거래 3,473",https://kream.co.kr/products/60301
4,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,"1,540,000원",2.6만,"5,346",거래 3.5만,https://kream.co.kr/products/38367


In [11]:
kream_detail.head()

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,"200,000",553558-053,-,GREY/WHITE/CREAM,"129,000원","202,000","201,000",https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,"1,555,000",DM7866-140,21/08/13,SAIL/BLACK/MILITARY BLUE,"189,000원","1,540,000","2,230,000",https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,"702,000",555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원","600,000","818,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,"312,000",DH6927-111,22/05/23,WHITE/BLACK/NEUTRAL GREY,"249,000원","299,000","313,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,"244,000",555088-108,22/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,"209,000원","207,000","270,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...


In [13]:
def num_preprocessing(data):
    if data == '':
        data = '0'
        
    if '.' in data:
        data = data.replace('만', '000')
    else:
        data = data.replace('만', '0000')
    
    data = data.replace('거래 ', '')
    data = int(data.replace('.', '').replace(',', ''))
    
    return data


def price_preprocessing(data):
    data = data.replace('-', '0')
    data = int(data.replace('원', '').replace(',', ''))
    
    return data


def release_price_preprocessing(data):
    filtered = re.findall('\d[0-9,]*원', data)
    if filtered:
        data = filtered[0].replace('원', '').replace(',', '')
    else:
        data = '0'
    
    return int(data)


def date_preprocessing(data):
    data = data.strip()
    if data != '-':
        if int(data[:2]) < 50:
            data = '20' + data
        elif int(data[:2]) >= 50:
            data = '19' + data
    
    data = data.replace('-', '1900/01/01')

    return data


def extract_collabo(data):
    p = re.compile(r"\bx\s\w+")
    filtered = p.findall(data)
    
    collabo = [word[2:] for word in filtered]
    
    return ','.join(collabo)

In [14]:
def kream_preprocessing(df):
    df = df.fillna('0')
    df['like'] = df['like'].apply(num_preprocessing)
    df['post'] = df['post'].apply(num_preprocessing)
    df['all_trade'] = df['all_trade'].apply(num_preprocessing)
    df['price'] = df['price'].apply(price_preprocessing)
    
    return df


def kream_detail_preprocessing(df):
    df['model_no'] = df['model_no'].apply(lambda x: x.strip())
    df['rescent_price'] = df['rescent_price'].apply(price_preprocessing)
    df['instant_buy_price'] = df['instant_buy_price'].apply(price_preprocessing)
    df['instant_sell_price'] = df['instant_sell_price'].apply(price_preprocessing)
    df['release_price'] = df['release_price'].apply(release_price_preprocessing)
    df['release_date'] = df['release_date'].apply(date_preprocessing)
    df['release_date'] = ps.to_datetime(df['release_date'], format='%Y/%m/%d')
    
    return df


def collabo_one_hot_encoding(df):
    df['collabo'] = df['_id'].apply(extract_collabo)
    df['Clot'] = df['collabo'].apply(lambda x: 1 if 'Clot' in x.split(',') else 0)
    df['Fragment'] = df['collabo'].apply(lambda x: 1 if 'Fragment' in x.split(',') else 0)
    df['Union'] = df['collabo'].apply(lambda x: 1 if 'Union' in x.split(',') else 0)
    df['Jayson Tatum'] = df['collabo'].apply(lambda x: 1 if 'Jayson' in x.split(',') else 0)
    df['J Balvin'] = df['collabo'].apply(lambda x: 1 if 'J' in x.split(',') else 0)
    df['League of Legends'] = df['collabo'].apply(lambda x: 1 if 'League' in x.split(',') else 0)
    df['Paris Saint Germain'] = df['collabo'].apply(lambda x: 1 if 'Paris' in x.split(',') else 0)
    df['A Ma Maniere'] = df['collabo'].apply(lambda x: 1 if 'A' in x.split(',') else 0)
    df['Trophy Room'] = df['collabo'].apply(lambda x: 1 if 'Trophy' in x.split(',') else 0)
    df['Travis Scott'] = df['collabo'].apply(lambda x: 1 if 'Travis' in x.split(',') else 0)
    df['Maison Chateau Rouge'] = df['collabo'].apply(lambda x: 1 if 'Maison' in x.split(',') else 0)
    df['Dior'] = df['collabo'].apply(lambda x: 1 if 'Dior' in x.split(',') else 0)
    df['Nina Chanel Abney'] = df['collabo'].apply(lambda x: 1 if 'Nina' in x.split(',') else 0)
    df['Billie Eilish'] = df['collabo'].apply(lambda x: 1 if 'Billie' in x.split(',') else 0)
    df['Off White'] = df['collabo'].apply(lambda x: 1 if 'Off' in x.split(',') else 0)
    df['SoleFly'] = df['collabo'].apply(lambda x: 1 if 'SoleFly' in x.split(',') else 0)
    df['Russell Westbrook'] = df['collabo'].apply(lambda x: 1 if 'Russell' in x.split(',') else 0)
    df['Supreme'] = df['collabo'].apply(lambda x: 1 if 'Supreme' in x.split(',') else 0)
    df['Levis'] = df['collabo'].apply(lambda x: 1 if 'Levis' in x.split(',') else 0)
    df['Patta'] = df['collabo'].apply(lambda x: 1 if 'Patta' in x.split(',') else 0)
    df['Aleali May'] = df['collabo'].apply(lambda x: 1 if 'Aleali' in x.split(',') else 0)
    df['Social Status'] = df['collabo'].apply(lambda x: 1 if 'Social' in x.split(',') else 0)
    df['Ghetto Gastro'] = df['collabo'].apply(lambda x: 1 if 'Ghetto' in x.split(',') else 0)
    df['Blue The Great'] = df['collabo'].apply(lambda x: 1 if 'Blue' in x.split(',') else 0)
    df['Lance Mountain'] = df['collabo'].apply(lambda x: 1 if 'Lance' in x.split(',') else 0)
    df['SNS'] = df['collabo'].apply(lambda x: 1 if 'SNS' in x.split(',') else 0)
    df['Facetasm'] = df['collabo'].apply(lambda x: 1 if 'Facetasm' in x.split(',') else 0)
    df['DSM'] = df['collabo'].apply(lambda x: 1 if 'DSM' in x.split(',') else 0)
    df['Kaws'] = df['collabo'].apply(lambda x: 1 if 'Kaws' in x.split(',') else 0)
    df['Olivia Kim'] = df['collabo'].apply(lambda x: 1 if 'Olivia' in x.split(',') else 0)
    df['Titan'] = df['collabo'].apply(lambda x: 1 if 'Titan' in x.split(',') else 0)
    df['Melody Ehsani'] = df['collabo'].apply(lambda x: 1 if 'Melody' in x.split(',') else 0)
    df['Anna Wintour'] = df['collabo'].apply(lambda x: 1 if 'Anna' in x.split(',') else 0)
    df['Converse'] = df['collabo'].apply(lambda x: 1 if 'Converse' in x.split(',') else 0)
    df['Vogue'] = df['collabo'].apply(lambda x: 1 if 'Vogue' in x.split(',') else 0)
    df['Serena Williams'] = df['collabo'].apply(lambda x: 1 if 'Serena' in x.split(',') else 0)
    df['Carmelo Anthony'] = df['collabo'].apply(lambda x: 1 if 'Carmelo' in x.split(',') else 0)
    df['Carhartt'] = df['collabo'].apply(lambda x: 1 if 'Carhartt' in x.split(',') else 0)
    df['Sheila Rashid'] = df['collabo'].apply(lambda x: 1 if 'Sheila' in x.split(',') else 0)
    df['Pendleton'] = df['collabo'].apply(lambda x: 1 if 'Pendleton' in x.split(',') else 0)
    df['Rox Brown'] = df['collabo'].apply(lambda x: 1 if 'Rox' in x.split(',') else 0)
    df['Nike'] = df['collabo'].apply(lambda x: 1 if 'Nike' in x.split(',') else 0)
    df['Comme des Garcons Homme'] = df['collabo'].apply(lambda x: 1 if 'Comme' in x.split(',') else 0)
    df['PSNY'] = df['collabo'].apply(lambda x: 1 if 'PSNY' in x.split(',') else 0)
    
    return df


def series_one_hot_encoding(df):
    df['OG'] = df['_id'].str.contains(pat= 'OG').astype(int)
    df['Retro'] = df['_id'].str.contains(pat='Retro').astype(int)
    df['SP'] = df['_id'].str.contains(pat='SP').astype(int)
    df['QS'] = df['_id'].str.contains(pat='QS').astype(int)
    df['SB'] = df['_id'].str.contains(pat='SB').astype(int)
    df['SE'] = df['_id'].str.contains(pat='SE').astype(int)
    df['Chicago'] = df['_id'].str.contains(pat='Chicago').astype(int)
    
    return df


def download_image(df):
    print('Image downloading...')
    
    df['img_path'] = df['model_no'].apply(lambda x: f'image/{x.replace("/", " ")}.png')
    
    for idx, row in tqdm(df.iterrows()):
        urlretrieve(row['img_url'], row['img_path'])
    
    return df
    

def calculate_color_ratio(df):
    print('Calculating the color ratio...')
    
    conn = pymysql.connect(user=user, passwd=pw, host=host, db=db_name)
    query = 'SELECT R, G, B, color FROM rgb_color'
    rgb_color_df = ps.read_sql(query, conn)
    rgb_color_df = rgb_color_df.groupby(['R', 'G', 'B'])['color'].agg(**{'color': lambda x: x.mode()[0]}).reset_index()
    
    color_lst = ['갈색', '검정', '고동', '남색', '노랑', '모카', '민트', '발블', '밝은회색', '베이지', '보라', '분홍',
                 '빨강', '살구', '아이보리', '연두', '자몽', '주황', '진한회색', '청록', '청회색', '초록', '파랑', 
                 '하늘', '황갈색', '황토', '회색', '흰색']
    color_df = ps.DataFrame(None, columns=['model_no'] + color_lst)
    
    for idx, row in tqdm(df.iterrows()):
        img_name = row['model_no'].replace('/', ' ')
        img = Image.open(f'image/{img_name}.png')
        img = img.resize((128, 128))

        img_arr = np.array(img)
        img_arr = np.reshape(img_arr, (-1, 4))

        img_df = ps.DataFrame(img_arr, columns=['R', 'G', 'B', 'alpha'])
        img_df = img_df.drop('alpha', axis=1)
        img_df = img_df[~((img_df['R'] == 0) & (img_df['G'] == 0) & (img_df['B'] == 0))]
        img_df = img_df.reset_index(drop=True)
        
        img_df = img_df.merge(rgb_color_df, on=['R', 'G', 'B'], how='left')
        
        color_cnt = img_df[img_df['color'].notna()][['R', 'color']].groupby('color').count().reset_index()
        color_cnt = color_cnt[color_cnt['R'] >= 80]
        color_cnt['model_no'] = row['model_no']
        color_cnt = color_cnt.pivot(index='model_no', columns='color', values='R').reset_index()
        color_df = ps.concat([color_df, color_cnt])
    
    color_df = color_df.fillna(0)
    df = df.merge(color_df, on='model_no', how='left')
    df[color_lst] = df[color_lst].div(df[color_lst].sum(axis=1), axis=0)
    
    return df

In [ ]:
def pipeline(kream, kream_detail):
    kream = kream_preprocessing(kream)
    kream_detail = kream_detail_preprocessing(kream_detail)
    kream_detail = collabo_one_hot_encoding(kream_detail)
    kream_detail = series_one_hot_encoding(kream_detail)
    kream_detail = download_image(kream_detail)
    kream_detail = calculate_color_ratio(kream_detail)
    
    return kream.merge(kream_detail, on='_id', how='inner')